In [1]:
import math
import openai
import os
import sklearn.metrics
import random
from tqdm import tqdm
from langchain.llms import OpenAI

In [2]:
import baseten
baseten.login("yiVOous9.2mjkPJSPdya6FUFbGKmXaLyUF6ZxTQYs")

INFO API key set.
INFO:baseten:API key set.


In [3]:
from task_utils import TASKS, load_data, load_prompt, generate_prompts

In [10]:
## your LLM stack goes here

# Falcon 7B

model = baseten.deployed_model_version_id('q957r6w')

def call_llm(prompt):
    output = model.predict({"prompt": prompt, "temperature": 0.0})
    print(output)
    output = output["data"]["generated_text"]
    result = output.rsplit(" ", maxsplit=1)[-1]
    return result
    

In [17]:
model.predict({'top_p': 0.75, 'prompt': "Meaning of life is", 'num_beams': 4, 'temperature': 0.1})

{'data': ['Meaning of life is a question that has been asked since the beginning of time. The meaning of life is a question that has been asked since the beginning of time. The meaning of life is a question that has been asked since the beginning of time. The meaning of life is a question that has been asked since the beginning of time. The meaning of life is a question that has been asked since the beginning of time. The meaning of life is a question that has been asked since the beginning of time. The meaning of life is a question that has been asked since the beginning of time. The meaning of life is a question that has been asked since the beginning of time. The meaning of life is a question that has been asked since the beginning of time. The meaning of life is a question that has been asked since the beginning of time. The meaning of life is a question that has been asked since the beginning of time. The meaning of life is a question that has been asked since the beginning of tim

In [11]:
def evaluate(tasks: list, tasks_dir: str):
    report = dict()
    for task in tqdm(tasks):
        train_df, test_df = load_data(task=task, tasks_dir=tasks_dir)
        prompt_template = load_prompt(prompt_name="base_prompt.txt", task=task, tasks_dir=tasks_dir)
        prompts = generate_prompts(prompt_template=prompt_template, data_df=train_df)
        report[task] = dict()
        targets = list()
        outputs = list()
        for prompt, data in zip(prompts, train_df.iterrows()):
            datapoint_id, data = data
            output = call_llm(prompt)
            output = output.strip()
            targets.append(data['answer'])
            outputs.append(output)
            success = output == data['answer']
            report[task][datapoint_id] = {
                'prompt': prompt,
                'generated_output': output,
                'correct_output': data['answer'],
                'success': output == data['answer']
            }
        report[task]['balanced_accuracy'] = sklearn.metrics.balanced_accuracy_score(targets, outputs)
    
    print('Balanced Accuracy:', sum([report[task]['balanced_accuracy'] if not math.isnan(report[task]['balanced_accuracy']) else 0 for task in tasks])/len(tasks))
    
    return report


In [12]:
len(TASKS)

162

In [13]:
tasks_dir = '../legalbench'

# tasks=random.sample(TASKS, 10)

tasks= [task for task in TASKS if task.startswith("cuad")]

In [14]:
len(tasks)

38

In [15]:
report = evaluate(tasks=tasks, tasks_dir=tasks_dir)

  0%|          | 0/38 [00:06<?, ?it/s]

{'data': ["Does the clause describe a license grant to a licensee (incl. sublicensor) and the affiliates of such licensee/sublicensor?\n\nClause: Supplier hereby grants Bank of America a nonexclusive, worldwide, irrevocable, perpetual license to: (a) any patents related to or necessary or desirable to use the Software to the extent such patents are now held, licensed to or hereafter acquired by Supplier, for the purpose of allowing Bank of America and its Affiliates and permitted assigns to install, copy, use, execute, modify, distribute (as necessary or useful for Bank of America and its Affiliates and permitted assigns to enjoy their rights as set forth in the Agreement), make, have made, enhance, improve and alter the Software (both in Object Code and Source Code form) as necessary to conduct Bank of America business in accordance with the terms and restrictions or this Section; (b) any Copyrights now held, licensed to or hereafter acquired by Supplier in the Software for the purpos

TypeError: list indices must be integers or slices, not str

In [ ]:
print('Balanced Accuracy:', sum([report[task]['balanced_accuracy'] if not math.isnan(report[task]['balanced_accuracy']) else 0 for task in tasks])/len(tasks))